### Jupyter notebook to configure an initial state of the simulation

In the following notebook, the user can configure and create the initial cells position for the simulation. It is necessary to specify which .xml file is used for the simulation and the center of each cell population.

In [4]:
import csv
import xml.etree.ElementTree as ET
import math
import os

In [5]:
def create_cells_on_sphere(cell_radius, shape_radius, shape_center, domain_size):
    cells = []
    xc = 0
    yc = 0
    zc = 0
    x_spacing = cell_radius * math.sqrt(3)
    y_spacing = cell_radius * 2
    z_spacing = cell_radius * math.sqrt(3)

    tempPoint = [0.0, 0.0, 0.0]

    for z in range(int(shape_center[2] - shape_radius), int(shape_center[2] + shape_radius), int(z_spacing)):
        for x in range(int(shape_center[0] - shape_radius), int(shape_center[0] + shape_radius), int(x_spacing)):
            for y in range(int(shape_center[1] - shape_radius), int(shape_center[1] + shape_radius), int(y_spacing)):
                
                tempPoint[0] = x + (zc % 2) * 0.5 * cell_radius
                tempPoint[1] = y + (xc % 2) * cell_radius
                tempPoint[2] = z

                if math.sqrt((tempPoint[0] - shape_center[0]) ** 2 + (tempPoint[1] - shape_center[1]) ** 2 + (tempPoint[2] - shape_center[2]) ** 2) < shape_radius:
                    cells.append(tuple(tempPoint[:]))

                yc += 1
            xc += 1
        zc += 1
        
    filtered_cells = []

    for position in cells:
        if (
            domain_size[0][0] <= position[0] <= domain_size[0][1] and
            domain_size[1][0] <= position[1] <= domain_size[1][1] and
            domain_size[2][0] <= position[2] <= domain_size[2][1]
        ):
            filtered_cells.append(position)

    return filtered_cells

def create_cell_disc_positions(cell_radius, disc_radius, centers, domain_size):
    cell_spacing = 0.95 * 2.0 * cell_radius
    x = 0
    y = 0
    x_outer = 0

    positions = []
    tempPoint = [0.0, 0.0, 0.0]

    n = 0
    while(y < disc_radius):
        x = 0.0
        if(n%2 == 1):
            x = 0.5 * cell_spacing
        x_outer = math.sqrt(disc_radius**2 - y**2)
        while(x < x_outer):
            tempPoint[0]= x + centers[0]
            tempPoint[1]= y + centers[1]
            tempPoint[2]= 0.0
            positions.append(tempPoint[:])
            if(math.fabs(y) > 0.01):
                tempPoint[0]= x + centers[0]
                tempPoint[1]= -y + centers[1]
                tempPoint[2]= 0.0
                positions.append(tempPoint[:])
            if(math.fabs(x) > 0.01):
                tempPoint[0]= -x + centers[0]
                tempPoint[1]= y + centers[1]
                tempPoint[2]= 0.0
                positions.append(tempPoint[:])
                if(math.fabs(y) > 0.01):
                    tempPoint[0]= -x + centers[0]
                    tempPoint[1]= -y + centers[1]
                    tempPoint[2]= 0.0
                    positions.append(tempPoint[:])
            x = x + cell_spacing
        
        y = y + (cell_spacing* (math.sqrt(3.0)/2))
        n = n + 1

    filtered_cells = []

    for position in positions:
        if (
            domain_size[0][0] <= position[0] <= domain_size[0][1] and
            domain_size[1][0] <= position[1] <= domain_size[1][1] and
            domain_size[2][0] <= position[2] <= domain_size[2][1]
        ):
            filtered_cells.append(position)

    return filtered_cells

In [6]:
def write_csv_file(positions, cell_type):
    if os.path.exists("./cells.csv"):
        with open("./cells.csv", mode='a', newline='') as file:
            writer = csv.writer(file)
            for point in positions:
                final_point = (*point, cell_type)
                writer.writerow(final_point)
    else:
        with open("./cells.csv", mode='w', newline='') as file:
            writer = csv.writer(file)
            for point in positions:
                final_point = (*point, cell_type)
                writer.writerow(final_point)
    print("CSV file created successfully.")
    return
    

In [7]:
def create_init_file(config_file, cell_types, centers, radii):
    defined_domain = []
    ID_list = []
    volume_list = []
    tree = ET.parse(config_file)
    root = tree.getroot()
    for neighbor in root.iter('cell_definition'):
        for cell_type in cell_types:
            if neighbor.attrib['name'] == cell_type:
                ID_list.append(int(neighbor.attrib['ID']))
                for elem in neighbor.iter('volume'):
                     volume_list.append(int(elem.find('total').text))
    if ID_list is []:
        print("no cell types found in this xml, check the name of the cell types defined in your config file")
        for neighbor in root.iter('cell_definition'):
            print(neighbor.attrib["name"])
        return

    domain = root.find("domain")
    dim =  domain.find("use_2D")
    defined_domain.append([int(domain.find("x_min").text), int(domain.find("x_max").text)])
    defined_domain.append([int(domain.find("y_min").text), int(domain.find("y_max").text)])
    defined_domain.append([int(domain.find("z_min").text), int(domain.find("z_max").text)])
    if (dim.text == "true"):
        print("using 2D")
        i=0
        for ID in ID_list:
            cell_radius = (3 * volume_list[int(i)] / (4 * math.pi))**(1/3)          
            positions = create_cell_disc_positions(cell_radius, radii[i], centers[i], defined_domain)
            write_csv_file(positions, int(ID))
            i += 1
    else:
        i=0
        for ID in ID_list:
            cell_radius = (3 * volume_list[int(i)] / (4 * math.pi))**(1/3)
            positions = create_cells_on_sphere(cell_radius, radii[i], centers[i], defined_domain)
            write_csv_file(positions, int(ID))
            i += 1

    return
        
    
            



In [11]:
config_file_cell_cycle = "../config/cell_cycle/PhysiCell_settings_cell_cycle_2D.xml"
config_file_differentiation = "../config/differentiation/PhysiCell_settings_2D_diff.xml"
# insert here the name of the configuration file

In [12]:
cell_types = ["default"] # insert here the cell types you want to include as string --> ex(2 cell types): ["T0", "dendritic_cell"]
centers = [[0, 0, 0]] # insert the center coordinate for each cell type --> ex(2 cell types for 3D simulation): [[50, 50, 10], [150, 150, 10]] 
radii = [30] # insert here the radius for each cell type --> ex(2 cell types): [25, 60]

In [13]:
create_init_file(config_file, cell_types, centers, radii)

using 2D
CSV file created successfully.
